<a href="https://colab.research.google.com/github/Rune123-citizen/Basic-Real-Time-Face-Detection-with-JavaScript/blob/main/IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ========================================
# PART 0: Setup Environment
# ========================================
!pip install -q transformers datasets evaluate torch matplotlib seaborn

import torch
import transformers
import datasets
import evaluate
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ Libraries installed")
print("PyTorch version:", torch.__version__)
print("GPU available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU only")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00
✅ Libraries installed
PyTorch version: 2.8.0+cu126
GPU available: True
GPU: Tesla T4


In [ ]:
from google.colab import files
uploaded=files.upload()

Saving IMDB Dataset.csv to IMDB Dataset.csv


In [ ]:
import pandas as pd

# Replace with your uploaded filename
df = pd.read_csv("IMDB Dataset.csv")
print(df.head())
print(df.columns)  # To see column names


                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
Index(['review', 'sentiment'], dtype='object')


In [ ]:
from datasets import Dataset, DatasetDict

# Rename columns if needed
df = df.rename(columns={"review": "text", "sentiment": "label"})

# Map sentiment strings to integers
sentiment_map = {"positive": 1, "negative": 0}
df["label"] = df["label"].map(sentiment_map)

# Convert to dataset
dataset = Dataset.from_pandas(df)

# Optional: train/test split
dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_data = dataset["train"]
test_data = dataset["test"]

print(train_data[0])

{'text': "The film disappointed me for many reasons: first of all the depiction of a future which seemed at first realistic to me was well-built but did only feature a marginal role. Then, the story itself was a weak copy of Lost in Translation. The Middle-Eastern setting, man with family meets new girl overseas, karaoke bar, the camera movements and the imagery - all that was a very bad imitation of the excellent Lost in Translation which had also credibility. This movie tries to be something brilliant and cultural: it is not. I wonder why Tim Robbins even considered doing this film!? The female main actress is awful - did she play the precog in Minority Report? And why do you have to show the vagina in a movie like this? Lost in Translation didn't have to show excessive love scenes. R-Rated just for this? This movie isn't even worth watching it from a videostore!", 'label': 0}


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)

tok_train = train_data.map(tokenize, batched=True)
tok_test = test_data.map(tokenize, batched=True)

# Remove the original text column
tok_train = tok_train.remove_columns(["text"])
tok_test = tok_test.remove_columns(["text"])

tok_train.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
tok_test.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import evaluate

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
    report_to="none"
)

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return accuracy.compute(predictions=preds, references=labels)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tok_train,
    eval_dataset=tok_test,
    tokenizer=tokenizer, # Keep tokenizer here as it's still used internally by Trainer
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/tmp/ipython-input-600464073.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.273600,0.221475,0.912400
2,0.149200,0.264069,0.924000


TrainOutput(global_step=5000, training_loss=0.2101381181716919, metrics={'train_runtime': 1911.2247, 'train_samples_per_second': 41.858, 'train_steps_per_second': 2.616, 'total_flos': 5298695946240000.0, 'train_loss': 0.2101381181716919, 'epoch': 2.0})

In [ ]:
trainer.evaluate()
trainer.save_model("distilbert-sentiment")


In [ ]:
tokenizer.save_pretrained("distilbert-sentiment")


('distilbert-sentiment/tokenizer_config.json',
 'distilbert-sentiment/special_tokens_map.json',
 'distilbert-sentiment/vocab.txt',
 'distilbert-sentiment/added_tokens.json',
 'distilbert-sentiment/tokenizer.json')

In [ ]:
from transformers import pipeline

# Load from the folder you saved
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="distilbert-sentiment",
    tokenizer="distilbert-sentiment"
)

# Quick test on some sample reviews
print(sentiment_pipeline("I really loved this product, amazing quality!"))
print(sentiment_pipeline("Worst experience ever, total waste of money."))


Device set to use cuda:0


[{'label': 'LABEL_1', 'score': 0.9964608550071716}]
[{'label': 'LABEL_0', 'score': 0.9970742464065552}]


In [ ]:
!pip install huggingface_hub

from huggingface_hub import notebook_login

# Log in to Hugging Face (it will give you a link + token)
notebook_login()
